## DICOM Reader

Utility tool to read from DICOM MR images and do the following:

- Save best slice image as NumPy file
- Generate and save binary segmentation masks as NumPy files
- Save Patient IDs with corresponding MUL values to CSV file

3000x speed improvement from previous version:

- Old version:    5m 24.9s
- This version:       0.1s

In [119]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import line
import os

In [120]:
# Replace paths
dicom_path = "../data/dicom_sagittal"
numpy_path = "../data"
dicomdir_path = os.path.join(dicom_path, "DICOMDIR")

In [121]:
dicomdir_dataset = pydicom.dcmread(dicomdir_path)

# Print out to inspect the DICOMDIR dataset contents
# print("---DATASET")
# print(dicomdir_dataset)

In [122]:
# Pre-process DICOM files to create an index mapping SOP Instance UIDs to file paths
def build_dicom_index_from_dicomdir(dicomdir_path):
    dicom_index = {}
    dicomdir_dataset = pydicom.dcmread(dicomdir_path)
    # print("---DATASET")
    # print(dicomdir_dataset)

    # Find the Directory Record Sequence
    directory_record_sequence = dicomdir_dataset.get((0x0004, 0x1220))
    # print("---SEQUENCE")
    # print(directory_record_sequence)

    count = 0
    # Iterate through the Directory Record Sequence
    for record in directory_record_sequence:
        if record.DirectoryRecordType == "IMAGE":
            # print("---RECORD IMAGE")
            # print(record)
            try:
                # Extract SOP Instance UID and file path
                sop_instance_uid = record.ReferencedSOPInstanceUIDInFile
                file_path = "/".join(record.ReferencedFileID)
                # print(count, sop_instance_uid, file_path)

                # Update the index
                dicom_index[sop_instance_uid] = file_path
                count += 1
            except Exception as e:
                print(e)
                continue

    return dicom_index


dicom_index = build_dicom_index_from_dicomdir(dicomdir_path)
# print(dicom_index)

In [123]:
print(f"Found {len(dicom_index)} images")

# Display first 10 items in the index
print({k: dicom_index[k] for k in list(dicom_index)[:10]})

Found 5412 images
{'1.3.6.1.4.1.14519.5.2.1.7310.5101.312330389535335770866588983111': '6b7cd4da/bda8e51e/c2f1c6/cf7e7a7c', '1.3.6.1.4.1.14519.5.2.1.7310.5101.333802086270498941264908040381': 'd80ecb0a/f8b2719c/2b869432/56c56320', '1.3.6.1.4.1.14519.5.2.1.7310.5101.283621197730149193024583410294': '8bc59f8e/6335b3a8/f4cbdfa9/9feccc62', '1.3.6.1.4.1.14519.5.2.1.7310.5101.144213892326406999649971852351': '3f7c7412/9276366/f22dfe5d/7794c962', '1.3.6.1.4.1.14519.5.2.1.7310.5101.303809583121707351811548851702': 'f3334896/9f2fb7ad/73df3279/2d69538a', '1.3.6.1.4.1.14519.5.2.1.7310.5101.296321892660061826002851919706': '9ca182ca/a6d72e01/4ea97dad/d2b03479', '1.3.6.1.4.1.14519.5.2.1.7310.5101.986346696701218551045922551695': 'b9a0a908/6ba90cb4/4f8c95b4/a0e85b2c', '1.3.6.1.4.1.14519.5.2.1.7310.5101.480251647910816497812513236881': '5058574e/3acc6801/6638a89a/28a3cc78', '1.3.6.1.4.1.14519.5.2.1.7310.5101.126902383080474274276725971993': '6d577d8c/279a522e/e22c2200/d0cfd98f', '1.3.6.1.4.1.14519.5.

In [125]:
# Function to search for GSPS objects in the DICOMDIR file
def find_gsps_objects_in_dicomdir(dicomdir_path):
    gsps_objects = []

    dicomdir_dataset = pydicom.dcmread(dicomdir_path)
    # print(dicomdir_dataset)

    # Find the Directory Record Sequence
    directory_record_sequence = dicomdir_dataset.get((0x0004, 0x1220))

    # Iterate through the Directory Record Sequence
    for record in directory_record_sequence:
        # print("---- RECORD ----")
        # print(record)
        if record.DirectoryRecordType == "PRESENTATION":
            try:
                # Extract SOP Class UID
                sop_class_uid = record.get((0x0004, 0x1510)).value
                # print(sop_class_uid)
                if sop_class_uid == "1.2.840.10008.5.1.4.1.1.11.1":
                    # print(f"\nFound GSPS object")

                    gsps_file_path = os.path.join(dicom_path, *record.ReferencedFileID)
                    # print(gsps_file_path)
                    gsps_ds = pydicom.read_file(gsps_file_path)
                    # print("---- GSPS DS ----")
                    # print(gsps_ds)
                    gsps_objects.append(gsps_ds)

            except Exception as e:
                print(e)
                continue

    return gsps_objects


# Find GSPS objects in the DICOMDIR file
gsps_objects = find_gsps_objects_in_dicomdir(dicomdir_path)
print(gsps_objects)

[Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 174
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Grayscale Softcopy Presentation State Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.25.67411208332485720666945439369926249395
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.3
(0002, 0013) Implementation Version Name         SH: 'dcm4che-5.30.0'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0016) SOP Class UID                       UI: Grayscale Softcopy Presentation State Storage
(0008, 0018) SOP Instance UID                    UI: 2.25.67411208332485720666945439369926249395
(0008, 0020) Study Date                          DA: '20110305'
(0008, 0023) Content Date            

In [126]:
print(f"Found {len(gsps_objects)} annotations")

Found 15 annotations


In [127]:
def get_mul_length(mask_lines, mode="both"):
    if mode == "right":
        mask_lines = [mask_lines[0]]
    elif mode == "left":
        mask_lines = [mask_lines[1]]

    length = 0
    for m_line in mask_lines:
        x1, y1 = m_line[0]
        x2, y2 = m_line[1]
        length += np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    if mode == "both":
        length /= 2
    return length

In [128]:
def save_image_with_annotations(gsps_dataset, image_dataset, plot=False):
    if plot:
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))  # 1 row, 3 columns

        # Plot the image
        axes[0].imshow(image_dataset.pixel_array, cmap=plt.cm.bone)

    # ADDED: list of lines in mask
    mask_lines = []

    # Extract the Graphic Annotation Sequence from the GSPS object
    graphic_annotation_sequence = gsps_dataset.get((0x0070, 0x0001), [])

    for annotation in graphic_annotation_sequence:
        # Process Graphic Object Sequence if present
        for graphic_object in annotation.get((0x0070, 0x0009), []):
            graphic_data = graphic_object.GraphicData
            # Assuming POLYLINE type for simplicity; you will need to adapt this based on the actual GraphicType
            if graphic_object.GraphicType == "POLYLINE":
                # POLYLINE should have pairs of points (x, y), so we reshape the flat list into a list of tuples
                polyline = list(zip(graphic_data[::2], graphic_data[1::2]))
                # print("polyline", polyline)
                # We then split these tuples into two lists for x and y coordinates
                x, y = zip(*polyline)

                # ADDED: append to mask_lines
                mask_lines.append(polyline)

                if plot:
                    axes[0].plot(x, y, linestyle="-", linewidth=2, color="red")

    patient_id = gsps_dataset.PatientName

    # ADDED: save image files
    image_file = os.path.join(
        numpy_path, f"images_best_slice/slice_{patient_id}.npy"
    )
    np.save(image_file, image_dataset.pixel_array)

    # ADDED: save mask files
    both_mask = get_image_mask_binary(
        image_dataset.pixel_array, mask_lines, mode="both"
    )
    right_mask = get_image_mask_binary(
        image_dataset.pixel_array, mask_lines, mode="right"
    )

    both_mask_file = os.path.join(
        numpy_path, f"masks_both/mask_both_{patient_id}_both.npy"
    )
    right_mask_file = os.path.join(
        numpy_path, f"masks_right/mask_right_{patient_id}.npy"
    )
    np.save(both_mask_file, both_mask)
    np.save(right_mask_file, right_mask)
    print("Saved image and mask files.\n")

    if plot:
        axes[1].imshow(both_mask, cmap=plt.cm.bone)
        axes[2].imshow(right_mask, cmap=plt.cm.bone)
        plt.show()

    num_slices = None
    best_slice = int(image_dataset.InstanceNumber)
    mul_avg = get_mul_length(mask_lines, mode="both")
    mul_right = get_mul_length(mask_lines, mode="right")

    row = patient_id, num_slices, best_slice, mul_avg, mul_right
    return row


def get_image_mask_binary(image_array, mask_lines, mode="both"):
    mask = np.zeros(image_array.shape, dtype="uint8")

    if mode == "right":
        mask_lines = [mask_lines[0]]
    elif mode == "left":
        mask_lines = [mask_lines[1]]

    for m_line in mask_lines:
        m_line_points = list(reversed(m_line[0])) + list(reversed(m_line[1]))
        m_line_points = [int(x) for x in m_line_points]
        rr, cc = line(*m_line_points)
        mask[rr, cc] = 1

    return mask

In [129]:
def find_referenced_image(gsps_dataset, dicom_index):
    referenced_sop_instance_uid = (
        gsps_ds.ReferencedSeriesSequence[0]
        .ReferencedImageSequence[0]
        .ReferencedSOPInstanceUID
    )
    if referenced_sop_instance_uid in dicom_index:
        # print("Found referenced image")
        # print("- SOP Instance UID:", referenced_sop_instance_uid)
        image_path = os.path.join(dicom_path, dicom_index[referenced_sop_instance_uid])
        return pydicom.dcmread(image_path, force=True)
    # print("Could not find referenced image")
    return None

rows = []

# Process GSPS objects
for gsps_ds in gsps_objects:
    # Find the referenced image dataset
    # print("Searching for referenced image of GSPS object...")
    image_ds = find_referenced_image(gsps_ds, dicom_index)
    # print(image_ds)

    try:
        if image_ds:
            # Save the image with annotations
            row = save_image_with_annotations(gsps_ds, image_ds, plot=False)
            rows.append(row)
            # Assuming we only want to plot the first matching image and annotation set
    except Exception as e:
        print(e)

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.

Saved image and mask files.



In [130]:
csv_path = "../data/info.csv"

import csv
with open(csv_path, "a") as f:
    writer = csv.writer(f)
    writer.writerows(rows)